In [44]:
import pandas as pd
import sys
from fi.client import Client
from fi.client import ModelTypes, Environments
import uuid
import time

sys.path.append("../../")
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [45]:
fi_client = Client(
    api_key="8e54ce886a6143a4b5845069e15ff4da",
    secret_key="11d0a3d5b4284c239b625db849d3e5a0"
  
)

In [46]:
from datasets import load_dataset

# load train/validation/test splits of individual subset
ragbench_hotpotqa = load_dataset("rungalileo/ragbench", "hotpotqa")

# load a specific split of a subset dataset
ragbench_hotpotqa = load_dataset("rungalileo/ragbench", "hotpotqa", split="test")

# load the full ragbench dataset
ragbench = {}
for dataset in ['covidqa']:
  ragbench[dataset] = load_dataset("rungalileo/ragbench", dataset)

In [63]:
context_data = ragbench['covidqa']["train"].to_pandas()[0:10]

In [41]:
def process_df_context(context_df):
    rag_row_list = []
    for idx,row in context_df.iterrows():
        context_list = list(row["documents"])
        context_list = [[context] for context in context_list]
        user_chat = {"role":"user","content":row["question"],"context":context_list}
        assistant_chat = {"role":"assistant","content":row["response"]}
        rag_row_list.append(user_chat)
        rag_row_list.append(assistant_chat)
    return rag_row_list

In [64]:
context_data

,id,question,documents,response,generation_model_name,annotating_model_name,dataset_name,documents_sentences,response_sentences,sentence_support_information,unsupported_response_sentence_keys,adherence_score,overall_supported_explanation,relevance_explanation,all_relevant_sentence_keys,all_utilized_sentence_keys,trulens_groundedness,trulens_context_relevance,ragas_faithfulness,ragas_context_relevance,gpt3_adherence,gpt3_context_relevance,gpt35_utilization,relevance_score,utilization_score,completeness_score
0,358,What role does T-cell count play in severe hum...,[Title: Emergent severe acute respiratory dist...,The T-cell count plays a crucial role in sever...,gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: Emergent severe acute respirator...","[[a, The T-cell count plays a crucial role in ...",[{'explanation': 'This sentence is supported b...,[],True,The response is generally supported by the pro...,Documents 0 and 1 contain useful information f...,"[0d, 0e, 0f, 1d, 1e, 1f]","[0d, 0e, 0f, 1d, 1e, 1f]",NaN,NaN,NaN,NaN,1.0,0.473684,0.368421,0.315789,0.315789,1.000000
1,457,What is MVO?,[Title: Gene Knockdowns in Adult Animals: PPMO...,MVO is not defined or mentioned in the provide...,gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: Gene Knockdowns in Adult Animals...","[[a, MVO is not defined or mentioned in the pr...",[{'explanation': 'The sentence correctly state...,[],True,The first sentence states that MVO is not defi...,Documents 2 and 3 contain information about MV...,"[2a, 2b, 2c, 2d, 2e, 3a, 3b, 3c, 3d, 3e]","[2a, 2b, 2c, 2d, 2e, 3a, 3b, 3c, 3d, 3e]",NaN,NaN,NaN,NaN,0.0,0.529412,0.352941,0.588235,0.588235,1.000000
2,1172,What have sero-surveys of MERS virus found?,"[Title: MERS coronavirus: diagnostics, epidemi...",Sero-surveys of MERS virus found no sign of ME...,gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: MERS coronavirus: diagnostics, e...","[[a, Sero-surveys of MERS virus found no sign ...",[{'explanation': 'This sentence is partly supp...,[a],False,The response as a whole is mostly supported by...,"Documents 0, 1, 2, and 3 contain useful inform...","[0d, 0e, 2d, 3d]","[0d, 0e, 2d]",NaN,NaN,NaN,NaN,0.0,0.173913,0.086957,0.173913,0.130435,0.750000
3,470,How were untreated MDA-MB-231 cells labeled?,[Title: Kinome-Wide siRNA Screening Identifies...,The untreated MDA-MB-231 cells were labeled wi...,gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: Kinome-Wide siRNA Screening Iden...","[[a, The untreated MDA-MB-231 cells were label...",[{'explanation': 'The sentence is fully suppor...,[],True,The response consists of one sentence making t...,Document 0 contains relevant information for a...,"[0a, 0d]",[0d],NaN,NaN,NaN,NaN,1.0,0.041667,0.041667,0.083333,0.041667,0.500000
4,518,What is the number of inhabitants of Reunion I...,[Title: Pandemic Influenza Due to pH1N1/2009 V...,"Reunion Island has a population of 850,000 inh...",gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: Pandemic Influenza Due to pH1N1/...","[[a, Reunion Island has a population of 850,00...",[{'explanation': 'This sentence claims that Re...,[],True,The response contains one claim: that Reunion ...,Document 3 contains the relevant information t...,[3b],[3b],NaN,NaN,NaN,NaN,1.0,0.055556,0.055556,0.055556,0.055556,1.000000
5,1685,What genome sequence was available for this st...,[Title: Complete viral RNA genome sequencing o...,The genome sequences for the study were availa...,gpt-3.5-turbo-1106,gpt-4o,covidqa_train,"[[[0a, Title: Complete viral RNA genome sequen...","[[a, The genome sequences for the study were a...",[{'explanation': 'The sentence claims that gen...,[],True,The response consists of a single sentence cla...,Document 0 and Document 1 have relevant inform...,"[0b, 0c, 1b, 1d]","[0b, 0c]",NaN,NaN,NaN,NaN,1.0,0.166667,0.125000,0.166667,0.083333,0.500000
6,1432,What is the destabilization is further compou...,[Title: Asymmetry in the Presence of Migration...,The destabilization is further compounded by s...,g

In [55]:
#Shuffling data
context_data["documents"][0:5]=context_data["documents"].sample(5)

In [57]:
rag_data = process_df_context(context_data)

In [61]:
def upload_data(rag_data):
    return fi_client.log(
  "rag-covidqa-test-v5",
  ModelTypes.GENERATIVE_LLM,
  Environments.PRODUCTION,
  "v1",
  # str(uuid.uuid4()),
  int(time.time()),
  {
    "chat_history": rag_data
  }
).result()

In [62]:
for i in range(0,10,2):
    upload_data(rag_data[i:i+2])

'Title: Emergent severe acute respiratory distress syndrome caused by adenovirus type 55 in immunocompetent adults in 2013: a prospective observational study\nPassage: Recent studies have shown that the immune system plays a crucial role in the clearance of HAdV viremia and survival of the host . Chen et al. reported that, in the acute phase of HAdV-55 infection, patients with severe disease may have high levels of dendritic cells and Th17 cells . In our study, the only patient who recovered from severe infection had higher T-cell counts. Three of the five patients had relatively low T-cell counts when admitted. Our results suggest that these three patients may have been relatively immunocompromised and that a lower T-cell count may be a risk'